# Azure OpenAI Responses API Sample

In [ ]:
file_path = "<FileName>"
model_name = "o3-pro"

from dotenv import load_dotenv
import os


load_dotenv()  # take environment variables


system_prompt ='''
あなたは優秀な医者です。
'''

user_content = '''
与えられた心電図（ECG）データを分析し、以下の情報を提供してください：
1. 心拍数（bpm）    
2. 心拍のリズム（正常、頻脈、徐脈など）
3. 異常な波形の有無（例：心房細動、心室性期外収縮など）
4. その他の重要な所見
'''

api_version = "2025-04-01-preview"
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")


In [ ]:
from IPython.display import Image, display

import os

import base64
f = open('051243f.jpg', 'rb')
data = f.read()
image_data = base64.b64encode(data).decode("utf-8")
image_format = os.environ.get("FILE_SUFFIX")
data_url = f"data:image/{image_format};base64,{image_data}"
f.close()
display(Image(data))


In [ ]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
)

In [ ]:
response = client.responses.create(
    model= model_name,
    instructions="質問は英語で来るが、回答は日本語ですること",
    input=[
        {
            "role": "system",
            "content":  {
                "type": "text",
                "text": system_prompt
            },
            "role": "user",
            "content": [ 
                {"type": "input_text", "text": user_content},
                {"type": "input_image", "image_url": data_url}
                ]
            }
    ]
)


In [ ]:
print(f"model:{response.model}")
print(f"output:\n{response.output[1].content[0].text}")
print(f"input_tokens:{response.usage.input_tokens}\noutput_tokens:{response.usage.output_tokens}\ntotal_tokens:{response.usage.total_tokens}")


model:o3-pro
output:
以下は、提示された12誘導心電図（25 mm/s, 1 mV=10 mm）を肉眼で確認したうえでの概略評価です。画質・表示時間が限られるため、おおよその所見としてご理解ください。

1. 心拍数（bpm）  
   ・II誘導のRR間隔はおよそ4.5～5マス（大マス）＝0.9～1.0 秒。  
   ・300 ÷ 4.8 ≒ 62 bpm（おおよそ60前後）。  
   → 軽度にゆっくりめだが正常範囲内。

2. 心拍のリズム  
   ・各拍に整ったP波が存在し、P波‐QRS‐T波の並びが一定。  
   ・RR間隔も規則的。  
   → 洞調律（正常洞調律）。頻脈・徐脈の基準外（60 bpm弱なので境界的だが臨床的には正常洞調律）。

3. 異常波形の有無  
   ・P波：鋭過ぎず、二峰性・鋸歯状なし（心房細動/粗動を示唆する所見なし）。  
   ・QRS：幅≒80 ms前後で狭い。異常Q波・Δ波・幅広いPVCなどはなし。  
   ・ST：各誘導で上昇・下降とも顕著な偏位なし（虚血・梗塞を示唆せず）。  
   ・T波：対称性で陰転化や高尖化はみられない。  
   → 期外収縮、心房細動、WPW、伝導ブロックなどの不整脈的異常波形は認めず。

4. その他の重要所見  
   ・軸：I、aVFともにR波優位→軸はおおむね正常範囲（−30°～+90°）。  
   ・PR間隔：約160 msで正常。  
   ・QT/​QTc：概算で正常域内。  
   ・電位・R波進展：前胸部誘導でV1→V5にかけて正常なR波増高。  
   ・病的高電位・低電位、電解質異常を示唆する波形変化もみられず。

総合評価  
・正常洞調律、心拍数≈60 bpm。  
・明らかな不整脈、虚血性変化、伝導障害、肥大所見などの異常は認めません。  
・臨床的には「正常心電図」と考えられる所見です。

※実際の診断は臨床症状・既往歴・連続モニタリングなどとあわせて医師が総合判断します。本結果は参考情報としてご活用ください。
input_tokens:1224
output_tokens:1014
total_tokens:2238
